In [1]:
# Módulos necesarios
import os
from dotenv import load_dotenv, dotenv_values

In [3]:
# Módulos de LangChain
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [4]:
# Carga de variables de entorno
load_dotenv()
config = dotenv_values(".env")

In [5]:
# Se configura la API Key
os.environ["GOOGLE_API_KEY"] = config["GOOGLE_API_KEY"]

sys_prompt = SystemMessage(content="Eres un asistente que responde siempre en español.")

# Se inicializa el modelo de lenguaje
model = init_chat_model(
    model="gemini-2.5-flash",
    model_provider="google_genai",
    sys_prompt=sys_prompt,
)

In [6]:
# Definición de una herramienta personalizada

@tool
def get_weather(location: str) -> str:
    """Obtiene el clima actual para una ubicación dada.

    Args:
        location (str): La ubicación para la cual se desea obtener el clima.
    """
    weather = "sunny"
    answer = f"Current weather in {location} is {weather}."
    return answer

In [7]:
# Lista de herramientas disponibles
l_tools = [get_weather]

# Vinculación de las herramientas al modelo
model_binded = model.bind_tools(l_tools)

# Creación del vector de mensajes secuenciales (conversación)
messages = []

# Mensaje inicial del usuario se agrega a la conversación
user_query = HumanMessage(
    content="¿Cuál es el clima actual en Buenos Aires?")
messages.append(user_query)

# Invocación del modelo con la herramienta integrada
# Se envía la conversación al modelo
# El modelo puede decidir llamar a la herramienta si es necesario
llm_answer = model_binded.invoke(messages)

# En caso de que el modelo haya decidido usar una herramienta,
# se ejectua la herramienta 
# y se agrega su respuesta a la conversación
for tool_call in llm_answer.tool_calls:
    for tool in l_tools:
        if tool.name == tool_call["name"]:
            tool_result = tool.invoke(tool_call)
            messages.append(tool_result)

# Finalmente, se envía nuevamente la conversación al modelo
llm_answer_final = model_binded.invoke(messages)
print(llm_answer_final.content)

El clima actual en Buenos Aires es soleado.

